In [1]:
import numpy as np


In [2]:
import requests

url_test = 'https://oda.ft.dk/api/Afstemning?$expand=Sagstrin/Sag&$filter=vedtaget eq true&$inlinecount=allpages'

r = requests.get(url_test)

sager = r.json()

In [3]:
#This function takes dict, extracts the next link, gets the new dict and appends it to the old dict

def get_all_pages(url):

    r = requests.get(url)

    sager = r.json()
    prev = r.json()

    while 'odata.nextLink' in prev:

        r = requests.get(prev['odata.nextLink'])
        sager['value'] += r.json()['value']
        prev = r.json()

    return sager

In [19]:
sager = get_all_pages(url_test)
sager.keys()

dict_keys(['odata.metadata', 'odata.count', 'value', 'odata.nextLink'])

In [20]:
#checking if all sager have the same keys
test = [len(sager['value'][i].keys()) for i in range(len(sager['value']))]
np.unique(test)

array([ 9, 10])

In [22]:
#they don't, so we check how many sager have missing keys
missing_keys = [i for i in range(len(sager['value'])) if len(sager['value'][i].keys()) != 10]
missing_keys

[0,
 1099,
 1100,
 1327,
 1404,
 1531,
 1532,
 1533,
 1542,
 1544,
 1550,
 1556,
 1559,
 1560]

In [23]:
#the above is only 14, so we remove those sager
clean_sager = [sag for sag in sager['value'] if len(sag) == 10]

In [55]:
clean_sager[0]['id']

2

In [57]:
#This function takes an id for a vote and collects all the votes for that vote

def get_all_votes(id):
    url = f'https://oda.ft.dk/api/Afstemning({id})/Stemme?$inlinecount=allpages'
    r = requests.get(url)
    votes = r.json()
    prev = r.json()
    while 'odata.nextLink' in prev:
        r = requests.get(prev['odata.nextLink'])
        votes['value'] += r.json()['value']
        prev = r.json()
    return votes


votes_test = get_all_votes(clean_sager[0]['id'])

In [61]:
len(votes_test['value'])

179

In [50]:
#takes stemme-id which is id in stemme and returns info on the specific politician

def get_politician(id):
    
        url = f'https://oda.ft.dk/api/Stemme({id})/Akt%C3%B8r?$expand=Parti'
    
        r = requests.get(url)
    
        return r.json()
        # we are only interested in typeid: 5, which is a person

test = get_politician(1)

test

{'odata.metadata': 'https://oda.ft.dk/api/$metadata#Akt%C3%B8r/@Element',
 'id': 158,
 'typeid': 5,
 'gruppenavnkort': None,
 'navn': 'Eigil Andersen',
 'fornavn': 'Eigil',
 'efternavn': 'Andersen',
 'biografi': '<?xml version="1.0" encoding="utf-16"?>\n<membersMember xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" tid="0">\n  <profession>Journalist</profession>\n  <shortname>SFEIAN</shortname>\n  <firstname>Eigil</firstname>\n  <lastname>Andersen</lastname>\n  <party>Socialistisk Folkeparti</party>\n  <title>Eigil Andersen (SF)</title>\n  <pictureMiRes>http://www.ft.dk/~/media/CV/Foto/20111/SF/Eigil_Andersen/Eigil_Andersenx133.jpg.ashx</pictureMiRes>\n  <PictureHiRes>http://www.ft.dk/~/media/CV/Foto/20111/SF/Eigil_Andersen/Eigil_Andersen.zip.ashx</PictureHiRes>\n  <address>Folketinget, Christiansborg</address>\n  <zip>1240</zip>\n  <city>København K</city>\n  <phoneFolketinget>+45 3337 4403</phoneFolketinget>\n  <email>eigil.andersen@